In [1]:
import requests
import os
import numpy as np
from elasticsearch import Elasticsearch

In [51]:
es = Elasticsearch(
hosts=["https://localhost:9200"],
basic_auth = ("elastic", "pbP0lDZ+pgqG*49D0WG6"),
    # ca_certs="/Users/wowsianka/Downloads/elasticsearch-8.4.3/config/certs/http_ca.crt")
    # ,
verify_certs = False)


/opt/homebrew/anaconda3/envs/RecSys/lib/python3.9/site-packages/elasticsearch/_sync/client/__init__.py:395: SecurityWarning: Connecting to 'https://localhost:9200' using TLS with verify_certs=False is insecure


In [52]:
es.ping()

/opt/homebrew/anaconda3/envs/RecSys/lib/python3.9/site-packages/urllib3/connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


True

In [53]:
es.info()

/opt/homebrew/anaconda3/envs/RecSys/lib/python3.9/site-packages/urllib3/connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


ObjectApiResponse({'name': 'C12243', 'cluster_name': 'elasticsearch', 'cluster_uuid': 'LInjBR9NS_yqalgLzfMxug', 'version': {'number': '8.4.3', 'build_flavor': 'default', 'build_type': 'tar', 'build_hash': '42f05b9372a9a4a470db3b52817899b99a76ee73', 'build_date': '2022-10-04T07:17:24.662462378Z', 'build_snapshot': False, 'lucene_version': '9.3.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

#### Define an ES analyzer for Polish texts containing:
   - standard tokenizer
    - synonym filter with the following definitions:
   -  kpk - kodeks postępowania karnego
   - kpc - kodeks postępowania cywilnego
   - kk - kodeks karny
   - kc - kodeks cywilny
   - Morfologik-based lemmatizer
  -  lowercase filter
  
Define an ES index for storing the contents of the legislative acts.

The analyzer parameter specifies the analyzer used for text analysis when indexing or searching a text field.

Index z analyzer z synonimami kodeksów, Morfologik lematyzator i lowercase filter:


In [59]:
es.indices.delete(index='example_index')


ObjectApiResponse({'acknowledged': True})

In [57]:
import warnings
warnings.filterwarnings('ignore')

In [60]:
request_body = {
   "settings":{
      "analysis":{
         "analyzer":{
            "my_analyzer":{ 
               "type":"custom",
               "tokenizer":"standard",
               "filter":[
                   'my_synonym', 'morfologik_stem', "lowercase"
               ]
            },
         
         },
         "filter":{
            "my_synonym":{
               "type":"synonym",
               "synonyms": [ 
                            "kpk => kodeks postępowania karnego",
                            "kpc => kodeks postępowania cywilnego",
                            "kk => kodeks karny",
                            "kc => kodeks cywilny"]
            }
         }
      }
   },
   "mappings":{
       "properties":{
          "data": {
             "type":"text",
             "analyzer":"my_analyzer"
          }
      }
   }
}
es.indices.create(index = "example_index", body = request_body)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'example_index'})

#### Load the data to the ES index.

In [61]:
file_names = []
for file in os.listdir('./ustawy'):
    file_names.append(file)

file_names.remove('.DS_Store')
file_names.remove('.ipynb_checkpoints')


In [64]:
for file in file_names:
    with open("ustawy" + '/' + file, 'r') as f:
        content = f.read()
        # es.create("example_index", file, {"data": data})
        doc = {'data': content}
        es.index(index = "example_index", id = file[:-4], body = doc)

In [65]:
es.count(index="example_index")['count']



1178

#### 6.Determine the number of legislative acts containing the word ustawa (in any form).

In [10]:
query = {"query":{
                "match": {
                    "data": "ustawa"
                }
            }
        }
res =  es.search(index="example_index", body=query)
res['hits']['total']




/var/folders/tk/5b6jb4s56hgfr9kt8prkrgfr0000gn/T/ipykernel_83265/3496860262.py:7: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  res =  es.search(index="example_index", body=query)


{'value': 1178, 'relation': 'eq'}

#### 7. Determine the number of occurrences of the word ustawa by searching for this particular form, including the other inflectional forms.

The size parameter only tells how many hits should be returned in the response, so if you specify size: 10000 and 200000 records match, you'll get 10000 matching documents in the result hits, but total will state 200000

In [37]:
query = {
    "query": {
        "match": {
            "data": "ustawa"
        }
    },
    "highlight": {
        "fields": {
            "data": {
                "number_of_fragments": 1000, #The maximum number of fragments to return.
                "fragment_size": 1 #The size of the highlighted fragment in characters. Defaults to 100.
            }
        }
     },
    "size": 2000 # The size parameter is the maximum number of hits to return
}
res =  es.search(index="example_index", body=query, request_timeout=30)


/var/folders/tk/5b6jb4s56hgfr9kt8prkrgfr0000gn/T/ipykernel_83265/2082252403.py:17: DeprecationWarning: Passing transport options in the API method is deprecated. Use 'Elasticsearch.options()' instead.
  res =  es.search(index="example_index", body=query, request_timeout=30)
/var/folders/tk/5b6jb4s56hgfr9kt8prkrgfr0000gn/T/ipykernel_83265/2082252403.py:17: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  res =  es.search(index="example_index", body=query, request_timeout=30)


In [38]:
counter = 0
for hit in res['hits']['hits']:
    counter += len(hit['highlight']['data'])
counter

24934

#### Determine the number of occurrences of the word ustaw by searching for this particular form, including the other inflectional forms.

wowsianka@C12243 ~> curl -XGET -u "elastic:pbP0lDZ+pgqG*49D0WG6" --cacert /Users/wowsianka/Downloads/elasticsearch-8.4.3/config/certs/http_ca.crt "https://localhost:9200/example_index/_analyze" -H 'Content-Type: application/json' -d' (RecSys)
                    {
                      "analyzer": "my_analyzer",
                      "text": "ustaw"
                    }'
{"tokens":[{"token":"ustawa","start_offset":0,"end_offset":5,"type":"<ALPHANUM>","position":0},{"token":"ustawić","start_offset":0,"end_offset":5,"type":"<ALPHANUM>","position":0}]}⏎


In [80]:
query7 = {
    "query": {
        "match": {
            "data": "ustawić"
        }
    },
    "highlight": {
        "fields": {
            "data": {
                "number_of_fragments": 1000, #The maximum number of fragments to return.
                "fragment_size": 1 #The size of the highlighted fragment in characters. Defaults to 100.
            }
        }
     },
    "size": 2000 # The size parameter is the maximum number of hits to return
}
res7 =  es.search(index="example_index", body=query7, request_timeout=30)
counter7 = 0
for hit in res7['hits']['hits']:
    counter7 += len(hit['highlight']['data'])
counter7

913

In [75]:
print(counter7 + counter)

25847


#### Determine the number of legislative acts containing the words kodeks postępowania cywilnego in the specified order, but in any inflection form.

In [76]:
query = {"query": {
                "match_phrase": {
                    "data": {
                        "query": "kodeks postępowania cywilnego"
                            }
                        }
                    }
            }

res =  es.search(index="example_index", body=query)
res['hits']['total']



{'value': 99, 'relation': 'eq'}

#### Determine the number of legislative acts containing the words wchodzi w życie (in any form) allowing for up to 2 additional words in the searched phrase.

In [78]:
query = {
    "query": {
        "match_phrase": {
            "data": {
                "query": "wchodzi w życie",
                "slop": 2
            }
        }
    }
}
res =  es.search(index="example_index", body=query)
res['hits']['total']



{'value': 1174, 'relation': 'eq'}

#### Determine the 10 documents that are the most relevant for the phrase konstytucja.

In [90]:
# Allows you to add one or more sorts on specific fields. 
# Each sort can be reversed as well. The sort is defined on a per field level, with special field name for _score to sort by score
query = {
    "query": {
        "match": {
            "data": {
                "query": "konstytucja",
                
            }
        }
    },
    "sort": [
        {
            "_score": {
                "order": "desc"
            }
        }
    ]
}

res = es.search(index='example_index', body=query)
for hit in res['hits']['hits']:
    print(f"Ustawa {hit['_id']} - score {hit['_score']}")

Ustawa 1997_629 - score 6.8817687
Ustawa 2000_443 - score 6.6758547
Ustawa 1997_604 - score 6.644713
Ustawa 1996_350 - score 6.6397324
Ustawa 1997_642 - score 6.2639
Ustawa 2001_23 - score 6.068482
Ustawa 1996_199 - score 5.9381456
Ustawa 1999_688 - score 5.86007
Ustawa 2001_1082 - score 5.4758835
Ustawa 1997_681 - score 5.4758835


#### Print the excerpts containing the word konstytucja (up to three excerpts per document) from the previous task.

In [97]:
query = {
    "query": {
        "match": {
            "data": {
                "query": "konstytucja",
                
            }
        }
    },
    "sort": [
        {
            "_score": {
                "order": "desc"
            }
        }
    ],
    
    "highlight": {
        "fields": {
            "data": {
                "number_of_fragments": 3, #The maximum number of fragments to return.
                # "fragment_size": 10 #The size of the highlighted fragment in characters. Defaults to 100.
            }
        }
     },
    "size": 200 # The size parameter is the maximum number of hits to return
}


res = es.search(index='example_index', body=query)


In [98]:
for hit in res['hits']['hits']:
    print(f"Ustawa {hit['_id']} - score {hit['_score']}")
    for fragment in hit['highlight']['data']:
        print(fragment)
        print("=="*7)

Ustawa 1997_629 - score 6.8817687
o zmianie ustawy konstytucyjnej o trybie przygotowania
           i uchwalenia <em>Konstytucji</em> Rzeczypospolitej
W ustawie  konstytucyjnej z  dnia 23 kwietnia 1992 r. o trybie przygotowania i 
uchwalenia <em>Konstytucji</em>
Do zgłoszenia projektu <em>Konstytucji</em> załącza się wykaz 
                obywateli popierających zgłoszenie
Ustawa 2000_443 - score 6.6758547
umowy międzynarodowej i nie wypełnia przesłanek określonych w art. 89
     ust. 1 lub art. 90 <em>Konstytucji</em>
międzynarodowej lub załącznika nie
     wypełnia przesłanek określonych w art. 89 ust. 1 lub art. 90 <em>Konstytucji</em>
co do zasadności wyboru
  trybu ratyfikacji umowy międzynarodowej, o którym mowa w art. 89 ust. 2
  <em>Konstytucji</em>
Ustawa 1997_604 - score 6.644713
Jeżeli Trybunał Konstytucyjny wyda orzeczenie o sprzeczności celów partii 
   politycznej z <em>Konstytucją</em>
Jeżeli Trybunał Konstytucyjny wyda orzeczenie o sprzeczności z <em>Konstytucją</em>
 